<a href="https://colab.research.google.com/github/haydenclose/Cloud_based_Oil_Detection/blob/main/Cloud_Based_Analysis_of_Oil_Spills_from_Wrecks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cloud Based Analysis of Oil Spills from Wrecks

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Set-up of workspace
Import Earth Engine API (`import ee`).
Additionally, import useful python packages from examplar scripts (need to remove redundant ones after)

In [1]:
import ee
!pip install geemap
import geemap
!pip install geopandas
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd                                                             # Useful package to read in csv's etc...
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell. Authorisation last a week.

In [2]:
# Trigger the authentication flow.. Only need to do once a week
# ee.Authenticate()

# Initialize the library
ee.Initialize()

## Interactive base map

The [`folium`](https://python-visualization.github.io/folium/)
library can be used to display `ee.Image` objects on an interactive
[Leaflet](https://leafletjs.com/) map. Folium has no default
method for handling tiles from Earth Engine, so one must be defined
and added to the `folium.Map` module before use.

The following cell provides an example of adding a method for handing Earth Engine
tiles and using it to display an elevation model to a Leaflet map

In [3]:
# Import the Folium library.
import folium                                                                   # Used for Interactive mapping

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,                                                                # Adds the layer name to the radio buttons rather than web address
    overlay = True,                                                             # Allow layers to overlay upon each
    control = True                                                              # Allows the user to turn on and off layers
  ).add_to(self)                                                                # Allows layers to be added after this base layer

# Add EE drawing method to folium, to be able to add other layers to this base map (see examplar below)
folium.Map.add_ee_layer = add_ee_layer                                         

from folium.plugins import Draw                                                 # Toolbox for adding the toolbar
draw = Draw(export=True)                                                        # Adds toolbars to folium plots

print('This sets up te base parameters of the map with no output')


This sets up te base parameters of the map with no output


Now lets make an examplar interactive map just with the base layer

In [4]:
lat, lon = 52, 0                                                                # Define the Lat and Lon to centre the map on (UK)
my_map = folium.Map(location=[lat, lon], zoom_start=7)                          # Create the map zoomed to level 7 on our defined Lat and Lon (not yet ouputted)

draw.add_to(my_map)                                                             # Add the draw toolbar to the map

# Display the map
display(my_map)

### Annotating map with points
We have a list of wreck locations (stored on CDP). I have uploaded the file called 'Wreck Database_V2.2.xls' to my google drive. 

Now we use pandas to read the excel file and the `Wreck.head()` to view the first few rows of the dataframe. Note the ***magic wand*** button that allows for the interaction with the data



In [5]:
Wrecks = pd.read_excel('Wreck Database_V2.3.xls')

Wrecks.head()

,Owner,Wreck_ID,Catergory,Latitude,Longitude,Depth,Cargo
0,MOD,SS DERBENT,Priority,53.473390,-4.235660,45m,3700 tons of fuel oil
1,MOD,HMS PRINCE OF WALES,Priority,3.517583,104.464383,~68m,NaN
2,MOD,HMS REPULSE,Priority,3.620619,104.345181,~68m,NaN
3,MOD,RFA WAR MEHTAR,Priority,52.605000,2.148330,Between 26.5 m and 40 m with 2-3 m scour.,7000 tonnes Admiralty fuel oil
4,MOD,RFA ATHELSTANE,Priority,7.332520,81.939600,42m,6096 tonnes Admiralty fuel oil


### Adding the wrecks to the map
Here we use a little `for` loop to go through each row of the wreck dataframe and catergorise by priority.

This method also adds the wrecks to the `Folium.FeatureGroup`  so we can turn on and off the wrecks we do and dont want in the feature control panel.

Finally we can add a `popup`, this can be an extensive table but for now to keep simple it is just the wrecks name.

In [6]:
Point_map = folium.Map(location=[lat, lon], zoom_start=7)                       # Base map with zoomed location and level.
for Wreck_group, Wreck in Wrecks.groupby('Catergory'):                          # For loop to group the wrecks together by priority
    feature_group = folium.FeatureGroup(Wreck_group)                            # Add the wreck groups to the feature control panel
    for row in Wreck.itertuples():                                              # Loop through the individual wrecks to plot
        folium.CircleMarker(location=[row.Latitude, row.Longitude],             # Adds circle marker by lat and long
                            popup=row.Wreck_ID,                                 # If click a point, tells us the name of the wreck
                            radius = 3,                                         # Size of point
                            fill_opacity=1).add_to(feature_group)          # Colour of outline by catergory
    feature_group.add_to(Point_map)                                             # Adds the points to the map 

folium.LayerControl().add_to(Point_map)                                         # Add the layer control to the map
Point_map                                                                       # Plots the map

# Using geemap
Geemap is an amazing package that wraps up alot of different functions into nice interactive mapping package.

See below for the basic output

*   Has tool bar on the left that can search by name/address, lat-lon or data
*   Draw toolbar to add polygons, points etc...
*   On the right have a little spanner that has a lot of useful tools see https://geemap.org/
*   At the top if click the symbol next to the spanner can toggle layers on and off and adjust transparency





In [45]:
BasicMap = geemap.Map()
BasicMap


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Add Points to geemap

Add ponts from a dataframe with a simple piece of code `Map.add_points_from_xy(df, x="", y="")`

This also add the data as a popup when clicked 





In [8]:
PointsGeeMap = geemap.Map()
PointsGeeMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")
PointsGeeMap

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Sentinel-1 imagery example
Example from https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD

In [10]:
imgVV = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select('VV')

def func_skh(image):
          edge = image.lt(-30.0)
          maskedImage = image.mask().And(edge.Not())
          return image.updateMask(maskedImage) \
        .map(func_skh)

desc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spring = ee.Filter.date('2015-03-01', '2016-05-20')
lateSpring = ee.Filter.date('2015-04-21', '2016-05-10')
summer = ee.Filter.date('2015-06-11', '2016-08-31')

descChange = ee.Image.cat(
        desc.filter(spring).mean(),
        desc.filter(lateSpring).mean(),
        desc.filter(summer).mean())

ascChange = ee.Image.cat(
        asc.filter(spring).mean(),
        asc.filter(lateSpring).mean(),
        asc.filter(summer).mean())

PointsGeeMap.setCenter(104.3, 3.6, 10)
PointsGeeMap.addLayer(ascChange, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
PointsGeeMap.addLayer(descChange, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)

PointsGeeMap

Map(bottom=710.0, center=[3.6, 104.3], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

<IPython.core.display.Javascript object>

# What next

In [18]:
k[0]

'A'

In [46]:
OrbitsMap = geemap.Map()

geometry = ee.Geometry.Polygon(
  [[[104.4, 3],
   [104.5, 3],
   [104.5, 4],
   [104.4, 4],
   [104.4, 3]]], None, False)


# Get Sentinel-1 data for an arbitrary 12 day period
s1 = (ee.ImageCollection('COPERNICUS/S1_GRD').
  filterDate('2020-05-01', '2020-05-13').
  filterMetadata('instrumentMode', 'equals', 'IW').
  filterBounds(geometry))

# Compose an ancillary property to categorize the images in this selection

def func_xmf(f):
  return f.set('platform_relorbit',
    ee.String(f.get('platform_number')).cat('_').
    cat(ee.Number(f.get('relativeOrbitNumber_start')).format('%.0f')).
    cat('_').cat(f.get('orbitProperties_pass')))

s1 = s1.map(func_xmf)

# Check which sensor/relative orbit combinations we have
orbits = ee.Dictionary(s1.aggregate_histogram('platform_relorbit'))

#print(orbits); # Check in console

keys = orbits.getInfo(); # Needed to loop

for k in keys:
  color = 'blue'
  if (k[0]=='B'):
    color = 'green'
  
  OrbitsMap.addLayer(ee.Image().paint(ee.FeatureCollection(s1).
    filterMetadata('platform_relorbit', 'equals', k), 0, 1),
    {'palette': [color]}, 'Footprint: ' + k, False)

for k in keys:
  OrbitsMap.addLayer(s1.filterMetadata('platform_relorbit', 'equals', k).first(),
    {'bands': ['angle'], 'min': 30, 'max': 45}, 'Incidence angle: ' + k, False)

OrbitsMap.addLayer(ee.Image().paint(geometry, 0, 1), {'palette': ['red']}, 'AOI', False)
OrbitsMap.centerObject(geometry, 6)
OrbitsMap


Map(center=[3.499911486769061, 104.45000000000044], controls=(WidgetControl(options=['position', 'transparent_…

In [25]:
BasicMap.user_roi.getInfo()

{'type': 'Polygon',
 'coordinates': [[[460.678711, -0.591878],
   [471.75293, -0.591878],
   [471.75293, 9.626476],
   [460.678711, 9.626476],
   [460.678711, -0.591878]]]}